## Create JSON annotations from YOLO Labels

In [ ]:
# prepare the data
!python src/faster_rcnn/data/prepare_data_faster_rcnn.py \
    --datasets src/faster_rcnn/data/center_stratified/C3 \
    --out-dir src/artifacts \
    --class-name polyp

## Run the model WITHOUT evaluation

In [ ]:
%cd /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures
%env PYTHONPATH=src:$PYTHONPATH

!python -m faster_rcnn.training.train_faster_rcnn \
    --train-json src/artifacts/train.json \
    --val-json src/artifacts/val.json \
    --roots-map src/artifacts/roots_map.json \
    --images-root src/faster_rcnn/data/center_stratified/C3 \
    --epochs 20 \
    --batch-size 8 \
    --num-workers 0 \
    --img-size 640 \
    --train-augs medium \
    --num-classes 2 \
    --freeze-backbone 2 \
    --opt sgd \
    --lr-backbone 1e-4 \
    --lr-heads 5e-3 \
    --weight-decay 1e-4 \
    --lr-scheduler none \
    --device cpu
    

## Run the Evaluation

In [ ]:
#!/usr/bin/env python3
from pathlib import Path
import sys
import torch

# Add your src folder to Python path
SRC = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders
from faster_rcnn.data.coco_eval_faster_rcnn import coco_map
from faster_rcnn.models.faster_rcnn_model import build_fasterrcnn

# --- paths ---
root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
ckpt_path  = root / "runs" / "frcnn_polyp" / "epoch_009.pth"
train_json = root / "src" / "artifacts" / "train.json"
val_json   = root / "src" / "artifacts" / "val.json"
roots_map  = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "src" / "faster_rcnn" / "data" / "center_stratified"

# --- setup model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = build_fasterrcnn(num_classes=2, pretrained=False)
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state["model"])
model.to(device).eval()

# --- build validation loader ---
_, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,        # affects TRAIN only; val uses build_val_augs
    train_augs="light",
    batch_size=2,
    num_workers=0,
)

# --- run COCO eval ---
metrics = coco_map(
    model,
    val_loader,
    device,
    coco_gt_json=str(val_json),
    verbose=True,     # prints COCO table at the end
    log_every=1,      # print per batch; set 5/10 if too chatty
)

print("\nFinal metrics:")
print(
    f"  mAP@[.5:.95] = {metrics['mAP_50_95']:.4f}\n"
    f"  mAP@0.5      = {metrics['mAP_50']:.4f}\n"
    f"  Recall       = {metrics['recall']:.4f}\n"
    f"  Precision    = {metrics['precision']:.4f}"
)
